In [82]:
def tic_tac():
    print('tic')
    yield
    print('tac')
    yield
    
    return 'Boum'

def spam():
    print('Spam')
    yield
    print('Eggs')
    yield
    print('Bacon')
    yield
    return 'SPAM!'

In [2]:
task = tic_tac()

In [4]:
while True:
    try:
        next(task)
    except StopIteration as stop:
        print('Return value:', repr(stop.value))
        break

Return value: None


In [5]:
def count():
    i = 0
    
    while True:
        print('tick')
        i += 1
        yield i

In [6]:
gen = count()

In [11]:
next(gen)

tick


5

In [12]:
def receiver():
    print('ready ....')
    while True:
        data = yield
        print('received', data)

In [13]:
gen = receiver()

In [16]:
next(gen)

received None


In [17]:
gen.send('test')

received test


In [27]:
x = 2
print('{!r}'.format(x))

2


In [56]:
from collections import deque

STATUS_NEW = 'NEW'
STATUS_RUNNING = 'RUNNING'
STATUS_FINISHED = 'FINISHED'
STATUS_ERROR = 'ERROR'

In [66]:
class Task:
    def __init__(self, coro):
        self.coro = coro
        self.name = coro.__name__
        self.status = STATUS_NEW
        self.return_value = None
        self.error_value = None
        
    def run(self):
        try:
            self.status = STATUS_RUNNING
            next(self.coro)
        except StopIteration as stop:
            self.status = STATUS_FINISHED
            self.return_value = stop.value
        except Exception as err:
            self.status = STATUS_ERROR
            self.error_value = err
            
    def is_done(self):
        return self.status in {STATUS_FINISHED, STATUS_ERROR}
    
    def __repr__(self):
        result = ''
        
        if self.is_done():
            result = '{}'.format(self.return_value or self.error_value)
        
        return '<Task "{}" [{}] {}>'.format(self.name, self.status, result)

In [95]:
class Loop:
    def __init__(self):
        self._running = deque()
    
    def _loop(self):
        task = self._running.popleft()
        task.run()
        if task.is_done():
            print(task)
            return
        self.schedule(task)
        
    def run_until_empty(self):
        while self._running:
            self._loop()
            
    def run_until_complete(self, task):
        task = self.schedule(task)
        while not task.is_done():
            self._loop()
            
    def schedule(self, task):
        if not isinstance(task, Task):
            task = Task(task)
        self._running.append(task)
        return task

In [96]:
event_loop = Loop()

In [97]:
event_loop.run_until_complete(spam())

Spam
Eggs
Bacon
<Task "spam" [FINISHED] SPAM!>


In [92]:
event_loop.run_until_empty()

tic
tac
<Task "tic_tac" [FINISHED] Boum>


In [98]:
import asyncio

In [102]:
event_loop = asyncio.get_event_loop()

In [105]:
event_loop.run_until_complete(asyncio.wait(tic_tac()))

RuntimeError: This event loop is already running